In [ ]:
# Kamiwaza AutoRAG: Data Ingestion and Retrieval Example
# 
# This notebook demonstrates how to use Kamiwaza's RAG system to:
# 1. Ingest documents and create vector embeddings
# 2. Perform semantic search
# 3. Generate LLM responses based on retrieved context

import os
import json
from glob import glob
from kamiwaza_client import KamiwazaClient

In [ ]:
# Initialize the client
client = KamiwazaClient("http://localhost:7777/api")

# Set up a collection name for our documents
COLLECTION_NAME = "product_documentation"

In [ ]:
openai_client = client.openai.get_client('Qwen2.5-Coder-32B-Instruct-GGUF')

In [ ]:
# ## 1. Document Ingestion Pipeline
# 
# First, we'll define functions to process documents, generate embeddings,
# and store them in the vector database.

def process_documents(file_paths, collection_name):
    """
    Process a set of documents, creating embeddings and storing them in the vector database.
    
    Args:
        file_paths (list): List of file paths to process
        collection_name (str): Name of the vector DB collection to use
    
    Returns:
        dict: Statistics about the processing results
    """
    print(f"Processing {len(file_paths)} documents...")
    
    # Initialize the embedder with contextual enhancement
    embedder = client.embedding.get_embedder(
        provider_type="contextual",
        model="nomic-ai/nomic-embed-text-v1.5",
        additional_config={
            "base_provider": "sentencetransformers",
            "llm_model_name": "qwen"  # Local LLM for context enhancement
        }
    )
    
    # Process each document
    total_chunks = 0
    processed_docs = 0
    
    for file_path in file_paths:
        # Read the file
        with open(file_path, 'r', encoding='utf-8') as f:
            text = f.read()
        
        # Get file metadata
        filename = os.path.basename(file_path)
        file_size = os.path.getsize(file_path)
        
        # Chunk the text with contextual enhancement
        chunk_response = embedder.chunk_text(
            text=text,
            max_length=510,  # Size that fits well within embedding model limits
            overlap=50,      # Overlap to maintain context between chunks
            return_metadata=True  # Get contextual descriptions
        )
        
        # Prepare chunks and metadata for storage
        chunks = chunk_response.chunks
        metadata_list = []
        
        for i, (chunk, offset, metadata) in enumerate(zip(
            chunks, 
            chunk_response.offsets, 
            chunk_response.metadata
        )):
            # Extract context description if available
            context_desc = ""
            if metadata and "context_description" in metadata:
                context_desc = metadata["context_description"]
            
            # Prepare metadata for this chunk
            metadata_list.append({
                "source": file_path,
                "filename": filename,
                "offset": offset,
                "chunk_index": i,
                "file_size": file_size,
                "additional_metadata": context_desc
            })
        
        # Generate embeddings for all chunks in the document
        embeddings = embedder.embed_chunks(chunks)
        
        # Store chunks and embeddings in vector database
        client.vectordb.insert(
            vectors=embeddings,
            metadata=metadata_list,
            collection_name=collection_name
        )
        
        # Update statistics
        total_chunks += len(chunks)
        processed_docs += 1
        
        print(f"Processed {filename}: {len(chunks)} chunks")
    
    return {
        "documents_processed": processed_docs,
        "total_chunks": total_chunks
    }

In [ ]:
# ## 2. Document Retrieval and Search
# 
# Now we'll define functions to search the vector database and retrieve relevant chunks.

def semantic_search(query, collection_name, limit=5):
    """
    Perform semantic search to find relevant document chunks.
    
    Args:
        query (str): The search query
        collection_name (str): Name of the vector DB collection to search
        limit (int): Maximum number of results to return
        
    Returns:
        dict: Search results including chunks and their metadata
    """
    print(f"Searching for: '{query}'")
    
    # Use the search service to find relevant chunks
    search_results = client.search.search_documents(
        query=query,
        collection_name=collection_name,
        limit=limit,
        retrieve_content=True,  # Include the actual content in results
        output_fields=[
            "source", "filename", "offset", "chunk_index", 
            "file_size", "additional_metadata"
        ]
    )
    
    # Process and return results
    results = {
        "query": query,
        "total_results": search_results["total_results"],
        "chunks": []
    }
    
    for chunk in search_results["chunks"]:
        results["chunks"].append({
            "content": chunk["content"],
            "score": chunk["score"],
            "filename": chunk["metadata"]["filename"],
            "offset": chunk["metadata"]["offset"],
            "context": chunk["metadata"].get("additional_metadata", "")
        })
    
    return results

In [ ]:
# ## 3. RAG Query with LLM Response Generation
# 
# Finally, we'll create a function that retrieves context and generates an LLM response.

def rag_query(query, collection_name, limit=5, model="qwen"):
    """
    Retrieve relevant chunks and generate an LLM response.
    
    Args:
        query (str): The user's question
        collection_name (str): Name of the vector DB collection to search
        limit (int): Maximum number of chunks to retrieve
        model (str): LLM model to use for response generation
        
    Returns:
        dict: Query results including LLM response and source chunks
    """
    # First, retrieve relevant chunks
    search_results = semantic_search(query, collection_name, limit)
    
    # If no results, return early
    if not search_results["chunks"]:
        return {
            "query": query,
            "response": "No relevant information found.",
            "chunks": []
        }
    
    # Prepare context from retrieved chunks
    contexts = []
    for i, chunk in enumerate(search_results["chunks"]):
        # Format each chunk with a reference number
        contexts.append(f"[{i+1}] {chunk['content']}")
    
    # Join contexts with spacing
    context_text = "\n\n".join(contexts)
    
    # Prepare prompt for the LLM
    prompt = f"""Answer the following question based on the provided context. 
Include citations using the numbers in brackets (e.g., [1], [2]).
If the context doesn't contain relevant information, just say so.

Context:
{context_text}

Question: {query}
"""
    
    # Get LLM client (using the OpenAI-compatible API)
    llm_client = client.openai.get_client(model)
    
    # Generate response
    response = llm_client.chat.completions.create(
        messages=[
            {"role": "system", "content": "You are a helpful assistant that answers questions based on provided context."},
            {"role": "user", "content": prompt}
        ],
        temperature=0.7,
        max_tokens=512
    )
    
    # Extract response text
    answer = response.choices[0].message.content
    
    return {
        "query": query,
        "response": answer,
        "chunks": search_results["chunks"]
    }

In [ ]:
# ## 4. Execution Example
# 
# Now let's run through a complete example

# First, we'll process our documents
document_paths = glob("./sample_docs/*.md")
if not document_paths:
    print("No documents found. Please ensure sample documents exist in ./sample_docs/")
else:
    # Process the documents
    stats = process_documents(document_paths, COLLECTION_NAME)
    print(f"Processed {stats['documents_processed']} documents with {stats['total_chunks']} total chunks")
    
    # Try a semantic search
    search_results = semantic_search(
        "How do I configure authentication settings?", 
        COLLECTION_NAME,
        limit=3
    )
    
    print("\nSearch Results:")
    print(f"Found {search_results['total_results']} results for '{search_results['query']}'")
    
    for i, chunk in enumerate(search_results["chunks"]):
        print(f"\n--- Result {i+1} (Score: {chunk['score']:.4f}) ---")
        print(f"Source: {chunk['filename']}")
        print(f"Context: {chunk['context'][:100]}...")
        print(f"Content: {chunk['content'][:150]}...")
    
    # Now try a RAG query
    print("\n\nRAG Query Example:")
    rag_results = rag_query(
        "What are the best practices for vector database optimization?",
        COLLECTION_NAME,
        limit=5
    )
    
    print(f"\nQuery: {rag_results['query']}")
    print("\nResponse:")
    print(rag_results["response"])
    
    print("\nSource Chunks:")
    for i, chunk in enumerate(rag_results["chunks"]):
        print(f"\n[{i+1}] {chunk['filename']} (Score: {chunk['score']:.4f})")